In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image

from threading import Thread

In [3]:
from ultralytics import YOLO

yolo = YOLO('model/yolo.pt')

In [4]:
import tensorflow as tf

In [5]:
model_gen = tf.keras.models.load_model('model/model_gen.h5')

In [6]:
width_standard= 1640
height_standard = 590
width = 512
height = 128

threshold = 25

In [7]:
def regon_of_interest(image):
    polygons = np.array([[(0, height_standard), (0, 300), (700, 240), (850, 240), (width_standard, 300), (width_standard, height_standard)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    mask = cv2.bitwise_and(image, mask)
    return mask

def crop_image(image):
    return np.array(Image.fromarray(image).crop((0, 240, width_standard, height_standard)))

In [8]:
def preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = regon_of_interest(img)
    img = crop_image(img)
    img = cv2.GaussianBlur(src=img, ksize=(5, 5), sigmaX=0, sigmaY=0)
    img = cv2.resize(img, (width, height))
    img = cv2.Canny(img, 50, 150)
    return img

In [9]:
def gen_image(img):
    processed_img = preprocess(img.copy())
    processed_img = processed_img.reshape(-1, height, width, 1)
    
    pred = model_gen(processed_img)
    
    return cv2.resize(pred.numpy()[0], (width_standard, height_standard - 240))

In [10]:
def object_detect(frame):
    pred = yolo.predict(frame)
    return pred[0].plot()
    # cv2.imshow('Frame', pred[0].plot())
    
def lane_detect(frame):
    img = gen_image(frame)
    img = img * 255
    
    img = np.where(img >= threshold, 255., 0)
    
    return img
    # cv2.imshow('frame', img)

In [12]:
cap = cv2.VideoCapture('example.mp4')
 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
 
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        
        cv2.imshow('object detect',object_detect(frame))
        cv2.imshow('lane detect',lane_detect(frame))
    
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        
    else: 
        break
 
cap.release()
cv2.destroyAllWindows()


0: 256x640 1 fishbone, 46.8ms
Speed: 0.0ms preprocess, 46.8ms inference, 6414.1ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 49.9ms
Speed: 2.0ms preprocess, 49.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 fishbone, 51.8ms
Speed: 6.6ms preprocess, 51.8ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 arrow, 1 fishbone, 49.2ms
Speed: 6.4ms preprocess, 49.2ms inference, 6.6ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 43.5ms
Speed: 8.1ms preprocess, 43.5ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 54.9ms
Speed: 0.0ms preprocess, 54.9ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 50.8ms
Speed: 0.0ms preprocess, 50.8ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 fishbone, 58.3ms
Speed: 0.0ms preprocess, 58.3ms inference

In [12]:
# Compile video from images

# import cv2
# import os

# image_folder = 'example'
# video_name = 'example.mp4'

# images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
# frame = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = frame.shape

# video = cv2.VideoWriter(video_name, 0, 30, (width,height))

# for image in images:
#     video.write(cv2.imread(os.path.join(image_folder, image)))

# cv2.destroyAllWindows()
# video.release()